In [1]:
import pandas as pd

# call detect-seq significant regions

In [2]:
# PATH = '../table/detect_seq.StatsTest.table_CTRL-VEGFA-Vector-PD_TREAT-GBEmini-AP-RNF2-All-PD_rep1.filtered_SMN-4_SCN-10_SMR-0.1_RPN-1_RTN-2.C2T_merge_G2A.tsv'
PATH = '../table/detect_seq.StatsTest.table_CTRL-VEGFA-Vector-PD_TREAT-GBEmini-dU-RNF2-All-PD_rep1.filtered_SMN-4_SCN-10_SMR-0.1_RPN-1_RTN-2.C2T_merge_G2A.tsv'
df = pd.read_csv(PATH,sep='\t')
print(df.shape)
df.head(2)

(18978, 17)


,chr_name,region_start,region_end,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR
0,chr1,187019,187019,18,16,0,0,0.129787,0.148238,0.000000,0.000000,"0,1 18,0 16,0",0.191765,NaN,NoTest,NaN,NaN
1,chr1,265421,265423,16,26,2,7,0.115366,0.240886,0.014421,0.064854,"0,1,2 14,0,2 19,3,4",1.062130,2.169045,TestOK,0.082913,0.306784


In [3]:
# {if($NF<0.05 && $13>1 && $10<3) print $0}
df_select = df[(df.p_value <= 0.005) & (df['ctrl_mut_count.norm'] < 3) & (df['log2_FC'] > 1)]
print(df_select.shape)
df_select.head(2)

(38, 17)


,chr_name,region_start,region_end,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR
338,chr1,116894504,116894504,11,18,3,13,0.079314,0.166767,0.021631,0.120443,"0,1 8,3 5,13",1.072184,2.477168,TestOK,0.003292,0.306784
432,chr1,124919388,124919388,9,20,3,14,0.064894,0.185297,0.021631,0.129708,"0,1 6,3 6,14",1.513693,2.584083,TestOK,0.001824,0.285094


In [4]:
df_bed = df_select[['chr_name','region_start','region_end']]
df_bed['middle'] = ((df_bed['region_start'] + df_bed['region_end'])/2).map(int)
df_bed.head(2)

<ipython-input-4-3ac48ad4d1a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bed['middle'] = ((df_bed['region_start'] + df_bed['region_end'])/2).map(int)


,chr_name,region_start,region_end,middle
338,chr1,116894504,116894504,116894504
432,chr1,124919388,124919388,124919388


In [5]:
df_bed['tss'] = df_bed['middle'] - 20
df_bed['tes'] = df_bed['middle'] + 20
df_bed.head()

<ipython-input-5-ba1481cf0ee6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bed['tss'] = df_bed['middle'] - 20
<ipython-input-5-ba1481cf0ee6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bed['tes'] = df_bed['middle'] + 20


,chr_name,region_start,region_end,middle,tss,tes
338,chr1,116894504,116894504,116894504,116894484,116894524
432,chr1,124919388,124919388,124919388,124919368,124919408
1660,chr2,94557172,94557172,94557172,94557152,94557192
1680,chr2,99224285,99224285,99224285,99224265,99224305
1779,chr3,16812877,16812877,16812877,16812857,16812897


In [6]:
df_bed = df_bed[['chr_name','tss','tes']]
df_bed.head(2)

,chr_name,tss,tes
338,chr1,116894484,116894524
432,chr1,124919368,124919408


In [7]:
print(PATH)

../table/detect_seq.StatsTest.table_CTRL-VEGFA-Vector-PD_TREAT-GBEmini-dU-RNF2-All-PD_rep1.filtered_SMN-4_SCN-10_SMR-0.1_RPN-1_RTN-2.C2T_merge_G2A.tsv


In [8]:
# attention! local path!
path_out = '/Users/mac/mac_data/3.projects/21.2020-11_1109_GBE_DETECT-Seq_new_protocol/igv-snapshot'
#
treat = 'GBEmini-dU-RNF2-All-PD_rep1'
ctrl = 'VEGFA-Vector-PD_rep1'


text = "maxPanelHeight 1500\nsnapshotDirectory {path}\n".format(path=path_out)

for chr_, tss, tes in df_bed.values.tolist():
    path_out_png = '293T-bat_TREAT-{treat}_CTRL-{ctrl}_{chr}_{tss}_{tes}.snapshot.png'.format(treat=treat,ctrl=ctrl,chr=chr_,tss=tss,tes=tes)
    text +="goto {chr}:{tss}-{tes}\nsnapshot {path_out_png}\n".format(chr=chr_,tss=tss,tes=tes,path_out_png=path_out_png)
print('\n'.join(text.split('\n')))
print('...\n')



path_out_batch = PATH.replace('tsv','snapshot_script.txt')
with open(path_out_batch,'w') as f:
    f.write(text)

maxPanelHeight 1500
snapshotDirectory /Users/mac/mac_data/3.projects/21.2020-11_1109_GBE_DETECT-Seq_new_protocol/igv-snapshot
goto chr1:116894484-116894524
snapshot 293T-bat_TREAT-GBEmini-dU-RNF2-All-PD_rep1_CTRL-VEGFA-Vector-PD_rep1_chr1_116894484_116894524.snapshot.png
goto chr1:124919368-124919408
snapshot 293T-bat_TREAT-GBEmini-dU-RNF2-All-PD_rep1_CTRL-VEGFA-Vector-PD_rep1_chr1_124919368_124919408.snapshot.png
goto chr2:94557152-94557192
snapshot 293T-bat_TREAT-GBEmini-dU-RNF2-All-PD_rep1_CTRL-VEGFA-Vector-PD_rep1_chr2_94557152_94557192.snapshot.png
goto chr2:99224265-99224305
snapshot 293T-bat_TREAT-GBEmini-dU-RNF2-All-PD_rep1_CTRL-VEGFA-Vector-PD_rep1_chr2_99224265_99224305.snapshot.png
goto chr3:16812857-16812897
snapshot 293T-bat_TREAT-GBEmini-dU-RNF2-All-PD_rep1_CTRL-VEGFA-Vector-PD_rep1_chr3_16812857_16812897.snapshot.png
goto chr4:50304977-50305017
snapshot 293T-bat_TREAT-GBEmini-dU-RNF2-All-PD_rep1_CTRL-VEGFA-Vector-PD_rep1_chr4_50304977_50305017.snapshot.png
goto chr4:5055

# call peaks and homer annotation

In [25]:
import os

In [17]:
# PATH_peaks = '../table/detect_seq.StatsTest.table_CTRL-VEGFA-Vector-PD_TREAT-GBEmini-dU-RNF2-All-PD_rep1.filtered_SMN-4_SCN-10_SMR-0.1_RPN-1_RTN-2.C2T_merge_G2A_call_peaks.tsv'
PATH_peaks = '../table/detect_seq.StatsTest.table_CTRL-VEGFA-Vector-PD_TREAT-GBEmini-AP-RNF2-All-PD_rep1.filtered_SMN-4_SCN-10_SMR-0.1_RPN-1_RTN-2.C2T_merge_G2A_call_peaks.tsv'

In [18]:
df = pd.read_csv(PATH_peaks,sep='\t')
df.head()

,chr_name,region_start,region_end,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR
0,chr1,113969,113969,0,14,0,14,0.000000,0.051374,0.0,0.051374,"0,1 0,0 0,14",-0.340716,1.082998,TestOK,0.6945,0.76619
1,chr1,183697,183697,15,52,0,0,0.108156,0.190819,0.0,0.000000,"0,1 15,0 52,0",0.819092,NaN,NoTest,NaN,NaN
2,chr1,183937,183937,16,36,0,0,0.115366,0.132105,0.0,0.000000,"0,1 16,0 36,0",0.195468,NaN,NoTest,NaN,NaN
3,chr1,186536,186536,11,20,0,0,0.079314,0.073392,0.0,0.000000,"0,1 11,0 20,0",-0.111961,NaN,NoTest,NaN,NaN
4,chr1,187019,187019,18,25,0,0,0.129787,0.091740,0.0,0.000000,"0,1 18,0 25,0",-0.500526,NaN,NoTest,NaN,NaN


In [19]:
df = df[df.p_value <= 0.005]
df.region_start = df.region_start - 1
df.region_end = df.region_end + 1
df.head()

,chr_name,region_start,region_end,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR
47,chr1,2652358,2652364,99,302,7,49,0.713829,1.108218,0.050473,0.179810,"0,1,2,3 92,7,0,0 252,49,1,0",0.634591,1.832897,TestOK,0.000606,0.155810
75,chr1,2655073,2655075,19,88,0,3,0.136998,0.322924,0.000000,0.011009,"0,1 16,3 76,12",1.237047,-1.139394,TestOK,0.000788,0.170672
76,chr1,2655104,2655106,16,70,2,7,0.115366,0.256872,0.014421,0.025687,"0,1 14,2 63,7",1.154826,0.832897,TestOK,0.003941,0.241086
121,chr1,2750669,2750671,6,52,0,6,0.043262,0.190819,0.000000,0.022018,"0,1 6,0 45,7",2.141020,-0.139394,TestOK,0.001863,0.207202
834,chr1,27679033,27679035,8,50,2,6,0.057683,0.183480,0.014421,0.022018,"0,1 6,2 44,6",1.669399,0.610505,TestOK,0.002873,0.228962


In [20]:
df_final = df[['chr_name','region_start','region_end']]
df_final

,chr_name,region_start,region_end
47,chr1,2652358,2652364
75,chr1,2655073,2655075
76,chr1,2655104,2655106
121,chr1,2750669,2750671
834,chr1,27679033,27679035
...,...,...,...
98235,chrY,56843227,56843229
98245,chrY,56844472,56844474
98249,chrY,56844910,56844912
98259,chrY,56846354,56846356


In [29]:
out_path = PATH_peaks.replace('tsv','filtered.bed')
df_final.to_csv(out_path,header=False,index=False,sep='\t')

In [ ]:
# annotatePeaks.pl table/detect_seq.StatsTest.table_CTRL-VEGFA-Vector-PD_TREAT-GBEmini-AP-RNF2-All-PD_rep1.filtered_SMN-4_SCN-10_SMR-0.1_RPN-1_RTN-2.C2T_merge_G2A_call_peaks.filtered.bed hg38 > temp.log